## Data Cleaning and Preparation
***

In [730]:
import pandas as pd
import numpy as np
import glob
import re

from bs4 import BeautifulSoup 
import requests
import pickle 

In [731]:
# Load in T100 data from csv files

files_li = glob.glob('zippedData/T_T100D*.2.csv') 
bts_df = pd.DataFrame(pd.read_csv(files_li[0])) 
for i in range(1,len(files_li)): 
    data = pd.read_csv(files_li[i]) 
    df1 = pd.DataFrame(data) 
    bts_df = pd.concat([df1,bts_df],axis=0, ignore_index=True) 

In [732]:
# Keep only top 40 used aircraft models

bts_40 = bts_df['AIRCRAFT_TYPE'].value_counts().head(40).index.to_list()
indexNum = bts_df[~bts_df['AIRCRAFT_TYPE'].isin(bts_40)].index
bts_df.drop(indexNum , inplace=True)

In [733]:
# Get list of top 40 aircraft names for data cleaning

bts_des = bts_df['AIRCRAFT_TYPE'].value_counts().to_frame()

In [734]:
# BTS uses number code to identify aircraft type in it's flight data. The legend is in a seperate csv file. 
# Load in the legend for aircraft type. 

ac_df = pd.read_csv('zippedData/L_AIRCRAFT_TYPE.csv')

In [735]:
# Merge flight data with aircfaft type. This will match BTS type names to the fligt data.

bts_des = pd.merge(bts_des,ac_df,left_on='AIRCRAFT_TYPE',right_on='Code')

In [736]:
common_words = ['BOEING','CHEROKEE', 'AIRBUS', 'INDUSTRIE', 'CANADAIR', 'CESSNA', 'CARAVAN', 'EMBRAER',
                'STATIONAIR','MCDONNELL', 'DOUGLAS', 'PIPER', 'PILATUS', 'DE', 'HAVILLAND', 'BEAVER', 'SUPER',
                'NAVAJO', 'EM', 'ER', 'BEECH']

In [737]:
common_words_eng = ['ILYUSHIN', 'ANTONOV', 'JUNKERS', 'LOCKHEED', 'BOEING','CHEROKEE', 'AIRBUS', 'INDUSTRIE', 
                    'CANADAIR', 'CESSNA', 'CARAVAN', 'EMBRAER', 'STATIONAIR','MCDONNELL', 'DOUGLAS', 'PIPER',
                    'PILATUS', 'DE', 'HAVILLAND', 'BEAVER', 'SUPER','NAVAJO', 'EM', 'ER', 'BEECH', 'CRAFT',
                    'BOMBARDIER','CANADA']

In [738]:
common_words_emi = ['TRENT', 'CFMI', 'PRATT & WHITNEY', 'GENERAL ELECTRIC', 'CANADA',
                    'ROLLSROYCE', 'LYCOMING', 'ALLISON', 'GARRETT', 'CONTINENTAL']

In [739]:
def short_name(name, x):
    if len(name) >= x:
        short_name = name[:x]
        return(short_name)
    return(name)

In [740]:
def remove_dash(name):
    name = name.replace('-', '')
    return(name)

In [741]:
def convert_upper(name):
    name = name.upper()
    return(name)

In [742]:
def convert_date(date):
    date = date.astype('datetime64[ns]')
    return(date)

In [743]:
def remove_slash(name):
    res = re.match(r"^[^/]*", name)
    return(str(res.group())) 

In [744]:
def remove_common(name, common_w):
    for common in common_w:
        name = name.replace(common, '')
    return(name)

In [745]:
def remove_space(name):
    name = name.replace(' ', "")
    return(name)

In [746]:

def convert_boeing(name):
    if re.match(r"^(7)(.)(7)", name):
        name = name[:3]
    return name

In [747]:
def convert_crj(name):
    if re.match(r"^RJ", name):
        name = name.replace('RJ', 'CRJ')
    return name

In [748]:
# C208 Conversion

In [749]:
bts_des.loc[bts_des['Code'] == 629, 'Description'] = 'Canadair RJ-200'
bts_des.loc[bts_des['Code'] == 631, 'Description'] = 'Canadair RJ-700'
bts_des.loc[bts_des['Code'] == 638, 'Description'] = 'Canadair RJ-900'
bts_des.loc[bts_des['Code'] == 838, 'Description'] = 'Boeing 737-800'
bts_des.loc[bts_des['Code'] == 674, 'Description'] = 'Embraer ERJ-130'
bts_des.loc[bts_des['Code'] == 676, 'Description'] = 'Embraer ERJ-140'
bts_des.loc[bts_des['Code'] == 675, 'Description'] = 'Embraer ERJ-145'
bts_des.loc[bts_des['Code'] == 677, 'Description'] = 'Embraer ERJ-170'
bts_des.loc[bts_des['Code'] == 678, 'Description'] = 'Embraer ERJ-190'

In [750]:
bts_des['Description'] = [convert_upper(x) for x in bts_des['Description']]
bts_des['Description'] = [remove_slash(x) for x in bts_des['Description']]
bts_des['Description'] = [remove_dash(x) for x in bts_des['Description']]
bts_des['Description'] = [remove_common(x, common_words) for x in bts_des['Description']]
bts_des['Description'] = [remove_space(x) for x in bts_des['Description']]
bts_des['Description'] = [short_name(x, 4) for x in bts_des['Description']]
bts_des['Description'] = [convert_boeing(x) for x in bts_des['Description']]
bts_des['Description'] = [convert_crj(x) for x in bts_des['Description']]

In [751]:
bts_df = pd.merge(bts_df,bts_des,left_on='AIRCRAFT_TYPE', right_on='Code')

In [752]:
bts_df.drop('count', axis= 1, inplace=True)

In [753]:
# Import Data Set from CSV

crash_df = pd.read_csv('zippedData/AviationData.csv', encoding='latin-1', dtype={6: str, 7: str, 14: str, 15: str,
                                                                                 28: str})

In [754]:
# Drop rows before 2012 so we can match data by year in bts_df
date_mask = (crash_df['Event.Date'] > '2011-12-31')

# THIS RETURNS ALL ROWS GREATER THAN THE DATE PROVIDED ABOVE
crash_df = crash_df.loc[date_mask]

# Rows without an aircraft model type don't help use. Dropping these rows.
crash_df = crash_df.dropna(subset=['Model'])

In [755]:
# Some values in Make are upper, and some are mixed case. Change the Make column to all uppercase
#crash_df['Make'] = crash_df['Make'].str.upper()

# Convert Event.Date from string to datetime. Used to filter df on years
crash_df['Event.Date'] = crash_df['Event.Date'].astype('datetime64[ns]')

In [756]:
crash_df['Model'] = [convert_upper(x) for x in crash_df['Model']]
#crash_df['Event.Date'] = [convert_date(x) for x in crash_df['Event.Date']]
crash_df['Model'] = [remove_slash(x) for x in crash_df['Model']]
crash_df['Model'] = [remove_dash(x) for x in crash_df['Model']]
crash_df['Model'] = [remove_common(x, common_words) for x in crash_df['Model']]
crash_df['Model'] = [remove_space(x) for x in crash_df['Model']]
crash_df['Model'] = [short_name(x, 4) for x in crash_df['Model']]
crash_df['Model'] = [convert_boeing(x) for x in crash_df['Model']]
#crash_df['Model'].replace('PA-', 'PA', inplace=True, regex=True)

In [757]:
def match_types(model):
    for newmodel in bts_des['Description']:
        if newmodel.startswith(model):
            #print(f'Processing: {model} \t:{newmodel}')
            return newmodel

In [758]:
crash_df['NewModel'] = [match_types(model) for model in crash_df['Model']]

## Engine Data

In [759]:
# In place of running this and making a web request, import the pickle file from below
# Assign URL |
#url = "https://asn.flightsafety.org/database/engines/"

# Make a GET request to fetch the raw HTML content 
#html_content = requests.get(url).text 

# Parse the response with html.parser
#soup = BeautifulSoup(html_content,"html.parser") 

# The data we need are wrapped in <a> tags. Grab all the <a> tags here and we'll filter later.
#datas = soup.find_all("a")


#Make list of URLs and list oF Aircraft to create dictionary
# k = []
# v = []
# for item in datas:
#     if 'engine' in item['href']:
#         k.append(item['href'])
#         v.append(item.text)

# lookup_dict = {'URL': k, 'Engine': v}


#url_df = pd.DataFrame.from_dict(lookup_dict)

# with open('lookup_dict.pkl', 'wb') as f:
#     pickle.dump(lookup_dict, f)

with open('zippedData/lookup_dict.pkl', 'rb') as f:
    lookup_dict = pickle.load(f)

In [760]:
# Take in the engine and retriev the matching data page with Aircraft data
# Add Aircraft list from web a python list.
# Return the python list

# url_p1 = 'https://asn.flightsafety.org'
# def lookup_ac(url_p2):
#     return_list = []
#     url = f'{url_p1}{url_p2}'
#     sub_content = requests.get(url).text
#     soup2 = BeautifulSoup(sub_content,"html.parser")
#     u_list=soup2.find_all('ul')[1]
#     i = 0 
#     while i < len(u_list.select('li')):
#         (u_list.select('li')[i].text)
#         return_list.append(u_list.select('li')[i].text)
#         i += 1
#     return(return_list)

# Create Aircraft column to list aircraft for each engine.
#url_df['Aircraft'] = url_df['URL'].map(lookup_ac)

# Save df to pickle file.
#url_df.to_pickle('zippedData/url_df.pkl')

with open('zippedData/url_df.pkl', 'rb') as f:
    url_df = pickle.load(f)

In [761]:
# Create a row for each aircraft for each engine. 
# Iterate through url_df['Aircraft'] column. For each element
# in list, create a new row.

engines = []
aircraft = []
for idx, row in url_df.iterrows():
    for ac in row['Aircraft']:
        engines.append(row['Engine'])
        aircraft.append(ac)

In [762]:
# Create dictionary from lists to load into dataFrame
ac_eng_dict = {'Engine': engines, 'Aircraft': aircraft}

In [763]:
# Create dataFrame from dictionary
engines_df = pd.DataFrame.from_dict(ac_eng_dict)

In [764]:
#engines_df['Engine'].unique()[51:100]

In [765]:
# df.loc[len(df.index)] = ['Amy', 89, 93] 
engines_df.loc[engines_df['Aircraft'] == 'Bombardier CRJ100 / 200 / 440'] = 'Canadair RJ-100'
engines_df.loc[engines_df['Aircraft'] == 'Bombardier CRJ700'] = 'Canadair RJ-700'
engines_df.loc[engines_df['Aircraft'] == 'Bombardier CRJ900'] = 'Canadair RJ-900'
engines_df.loc[len(engines_df.index)] = ['General Electric CF34', 'Canadair RJ-200']
engines_df.loc[len(engines_df.index)] = ['General Electric CF34', 'Canadair RJ-440']
engines_df.loc[len(engines_df.index)] = ['General Electric CF34', 'Embraer ERJ-130']
engines_df.loc[len(engines_df.index)] = ['General Electric CF34', 'Embraer ERJ-135']
engines_df.loc[len(engines_df.index)] = ['General Electric CF34', 'Embraer ERJ-140']
engines_df.loc[len(engines_df.index)] = ['General Electric CF34', 'Embraer ERJ-175']
engines_df.loc[len(engines_df.index)] = ['General Electric CF34', 'Embraer ERJ-190']
engines_df.loc[len(engines_df.index)] = ['Pratt & Whitney JT8D', 'McDonnell Douglas DC-9-80']
engines_df.loc[len(engines_df.index)] = ['Lycoming O-540', 'Piper PA-31']
engines_df.loc[len(engines_df.index)] = ['Lycoming O-540', 'Piper PA-32']
engines_df.loc[len(engines_df.index)] = ['Continental O-470', 'C206']
engines_df.loc[len(engines_df.index)] = ['Continental O-470', 'C208']
engines_df.loc[len(engines_df.index)] = ['Pratt & Whitney Canada PT6', 'Pilatus PC-12']
engines_df.loc[len(engines_df.index)] = ['Pratt & Whitney Canada PT6', 'DHC2']

In [766]:
engines_df['Aircraft'] = [convert_upper(x) for x in engines_df['Aircraft']]
engines_df['Aircraft'] = [remove_slash(x) for x in engines_df['Aircraft']]
engines_df['Aircraft'] = [remove_dash(x) for x in engines_df['Aircraft']]
engines_df['Aircraft'] = [remove_common(x, common_words_eng) for x in engines_df['Aircraft']]
engines_df['Aircraft'] = [remove_space(x) for x in engines_df['Aircraft']]
engines_df['Aircraft'] = [short_name(x, 4) for x in engines_df['Aircraft']]
engines_df['Aircraft'] = [convert_boeing(x) for x in engines_df['Aircraft']]
engines_df['Aircraft'] = [convert_crj(x) for x in engines_df['Aircraft']]

In [767]:
engines_df['Engine_Name'] = [convert_upper(x) for x in engines_df['Engine']]
engines_df['Engine_Name'] = [remove_slash(x) for x in engines_df['Engine_Name']]
engines_df['Engine_Name'] = [remove_dash(x) for x in engines_df['Engine_Name']]
engines_df['Engine_Name'] = [remove_common(x, common_words_emi) for x in engines_df['Engine_Name']]
engines_df['Engine_Name'] = [remove_common(x, common_words_emi) for x in engines_df['Engine_Name']]
engines_df['Engine_Name'] = [remove_space(x) for x in engines_df['Engine_Name']]
engines_df['Engine_Name'] = [short_name(x, 5) for x in engines_df['Engine_Name']]
#engines_df['Engine_Name'] = [convert_boeing(x) for x in engines_df['Engine_Name']]
#engines_df['Engine_Name'] = [convert_crj(x) for x in engines_df['Engine_Name']]

In [768]:
bts_des2 = pd.merge(bts_des, engines_df,left_on='Description', right_on='Aircraft', how='left')

In [769]:
bts_des2.drop_duplicates(inplace=True)

## Emissions Data

In [770]:
emissions_df = pd.read_excel('zippedData/edb-emissions-databank_v30__web_.xlsx',
                             sheet_name='Gaseous Emissions and Smoke', 
                             usecols=('A:D, AJ'))

In [771]:
emissions_df['Engine Identification'] = [convert_upper(x) for x in emissions_df['Engine Identification']]
emissions_df['Engine Identification'] = [remove_slash(x) for x in emissions_df['Engine Identification']]
emissions_df['Engine Identification'] = [remove_dash(x) for x in emissions_df['Engine Identification']]
emissions_df['Engine Identification'] = [remove_common(x, common_words_emi) for x in emissions_df['Engine Identification']]
emissions_df['Engine Identification'] = [remove_space(x) for x in emissions_df['Engine Identification']]
emissions_df['Engine Identification'] = [short_name(x, 5) for x in emissions_df['Engine Identification']]

In [772]:
def match_engine(str1):
    tmp_df = pd.DataFrame()
    mask = tmp_df['engine_id_starts_with_CFM'] = list( 
        map(lambda x: x.startswith(str1), emissions_df['Engine Identification'])) 
    filtered_df = emissions_df[mask]
    #return(filtered_df.at[0,1]['CO Dp/Foo Avg (g/kN)']).head(1)
    if filtered_df.shape[0] < 1:
        return(0)
    return(filtered_df.iat[1,4])

In [773]:
xyz = match_engine('CF34')

In [774]:
xyz

91.2

In [775]:
bts_des2['CO2'] = [match_engine(x) for x in bts_des2['Engine_Name']]

In [776]:
bts_des2[0:50]

,count,Code,Description,Engine,Aircraft,Engine_Name,CO2
0,346174,614,737,CFMI CFM56,737,CFM56,67.20
7,346174,614,737,CFMI LEAP-1B,737,LEAP1,20.71
8,346174,614,737,Pratt & Whitney JT8D,737,JT8D,130.60
10,291519,694,A320,CFMI CFM56,A320,CFM56,67.20
11,291519,694,A320,CFMI LEAP-1B,A320,LEAP1,20.71
12,291519,694,A320,IAE V2500,A320,IAEV2,0.00
13,291519,694,A320,Pratt & Whitney PW1000G,A320,PW100,0.00
14,240218,612,737,CFMI CFM56,737,CFM56,67.20
21,240218,612,737,CFMI LEAP-1B,737,LEAP1,20.71
22,240218,612,737,Pratt & Whitney JT8D,737,JT8D,130.60
